In [1]:
from module.source.jqdata import JQData
from jqdatasdk import finance
from sqlalchemy.orm.query import Query
from datetime import datetime
import dateutil
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
import pandas as pd

In [125]:
from utils.db import DBManager

In [126]:
db = DBManager()

In [2]:
jq = JQData()

auth success 


In [403]:
from jqdatasdk import finance 
df = finance.run_query(Query(finance.STK_EL_TOP_ACTIVATE)
                  .filter(
                      finance.STK_EL_TOP_ACTIVATE.day>="20200401" ))
df = df[df['link_name'].isin(['沪股通','深股通'])]
df['name'] = df['name'].map(lambda x: x.strip())
df['total']  =df['total'] / 10e7
df['day'] = df['day'].map(lambda x : x.strftime('%Y%m%d'))

In [207]:
import pyecharts.options as opts
from pyecharts.charts import HeatMap

In [5]:
from sqlalchemy.sql import or_

In [131]:
import os
from config import PROJECT_DIR

stock_pool = pd.read_csv(os.path.join(PROJECT_DIR, "config/pool.txt"), dtype=str,sep=' ',
                             names=['tag', 'name', 'code', 'price'])
stocks = stock_pool['code'].map(lambda x: jq.normalize_code(x)).tolist()
link_id = [310001 , 310002]
df=finance.run_query(Query(finance.STK_HK_HOLD_INFO).filter(
    or_(finance.STK_HK_HOLD_INFO.link_id==i  for i in link_id ),
    or_(finance.STK_HK_HOLD_INFO.code==i  for i in stocks ),
    finance.STK_HK_HOLD_INFO.day >='20200301'
))[['day','name','share_ratio']]
df['day'] = df['day'].map(lambda x : datetime.strftime(x,"%Y%m%d"))

trade_day = finance.run_query(Query(finance.STK_EXCHANGE_LINK_CALENDAR).filter(
    finance.STK_EXCHANGE_LINK_CALENDAR.day>='20200301',
    or_(finance.STK_EXCHANGE_LINK_CALENDAR.link_id==i  for i in link_id ))
)[['day','type']].drop_duplicates()
trade_day['day'] = trade_day['day'].map(lambda x : datetime.strftime(x,"%Y%m%d"))

merge_df = df.merge(trade_day , on='day',how='left')
merge_df = merge_df[merge_df['type'] == '正常交易日']
del merge_df['type']

result_df = merge_df.set_index(['day','name']).unstack('name')['share_ratio']

rs_df = ((result_df - result_df.shift()).fillna(0) * 100).applymap(lambda x : round(x))
selected = sorted(list(rs_df.iloc[-1].items()),key=lambda x: x[1],reverse=True)[0][0]

In [190]:
import pyecharts.options as opts
from pyecharts.charts import Line

c = Line()
c = c.add_xaxis(rs_df.index.tolist())

for key in rs_df:
    is_selected = key == selected
    c = c.add_yaxis(key, rs_df[key].values.tolist(), is_smooth=True,is_selected=is_selected)
    
c = c.set_series_opts(
        areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
        label_opts=opts.LabelOpts(is_show=False),
    )
c = c.set_global_opts(
        title_opts=opts.TitleOpts(title="北上资金流入流出"),
        xaxis_opts=opts.AxisOpts(
            axistick_opts=opts.AxisTickOpts(is_align_with_label=True),
            is_scale=False,
            boundary_gap=False,
        ),
    legend_opts=opts.LegendOpts(pos_left=100,pos_top=50)
    )
c.render_notebook()

In [173]:
c = c.set_global_opts(
        title_opts=opts.TitleOpts(title="北上资金流入流出"),
        xaxis_opts=opts.AxisOpts(
            axistick_opts=opts.AxisTickOpts(is_align_with_label=True),
            is_scale=False,
            boundary_gap=False,
        ),
    legend_opts=opts.LegendOpts(pos_left=100,pos_top=50)
    )
c.render_notebook()

In [193]:
from shapely.geometry import Polygon

In [194]:
a = Polygon([[121.45027452837054,31.246320269201878],[121.45009644425136,31.246574161383606],[121.45007558756414,31.246609299507277],[121.45007397968762,31.24665407559658],[121.45027526477399,31.247208666147287],[121.45028809647644,31.247227885697882],[121.4503113573435,31.247236732983357],[121.45113752623625,31.247326529834684],[121.45116479878075,31.247313796789474],[121.4511744261483,31.247289829289677],[121.45120894127409,31.24697644966102],[121.45121535986684,31.246955673519864],[121.45122418431741,31.246942899175245],[121.45124584206724,31.246938149986246],[121.45141108098841,31.24693692232881],[121.45143033268506,31.24692737007085],[121.45144236669574,31.24690580704208],[121.45145841906523,31.246774703103977],[121.45146243322496,31.24672833327269],[121.45145042962959,31.246338883975618],[121.45144401732806,31.246276497926488],[121.4514263756759,31.246214086679792],[121.45139429457085,31.24615324211183],[121.45131729523968,31.24608110167715],[121.4512451074003,31.246026564292897],[121.45098202210818,31.245882040478932],[121.4507429991509,31.24577915417004],[121.45069326728817,31.245787039881236],[121.45065957774084,31.24580295773625],[121.45027452837054,31.246320269201878]])

In [197]:
a.area?

In [198]:
Polygon.area?